In [93]:
%matplotlib qt
import time
import sys
import math
import numpy as np
import pandas as pd
from datetime import datetime
import calendar
import holidays
import matplotlib.pyplot as plt
import cPickle as pickle
from datetime import datetime
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution1D
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_friedman1
from sklearn.ensemble import GradientBoostingRegressor
sys.path.insert(0, '/Users/kenleejr92/energy_market_project/scripts/Feature_Selection/')
from Feature_Selection import LMP_Query
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_friedman1
from sklearn.ensemble import GradientBoostingRegressor

def string_to_date(string_date):
    return datetime.strptime(string_date, "%Y-%m-%d %H")

def date_to_string(date):
    return date.strftime("%Y-%m-%d %H")

def weekday_of_date(date):
    w = calendar.day_name[date.weekday()]
    if w == 'Sunday':
        w=0
    elif w == 'Monday':
        w=1
    elif w == 'Tuesday':
        w=2
    elif w == 'Wednesday':
        w=3
    elif w == 'Thursday':
        w=4
    elif w == 'Friday':
        w=5
    elif w == 'Saturday':
        w=6
    return w

def work_day_or_holiday(date):
    us_holidays = holidays.UnitedStates()
    if date in us_holidays or weekday_of_date(date) == "Sunday" or weekday_of_date(date) == "Saturday":
        return int(1)
    else: return int(0)

In [2]:
exemplars = None
with open('../Feature_Selection/exemplars.pkl', 'r') as f:
    exemplars = pickle.load(f)
qcrr = LMP_Query()
df=qcrr.query(exemplars)

In [126]:
mat = df.as_matrix()
results = []
lag = 24
for idx, e in enumerate(exemplars):
    model = None
    selected = None
    with open('../Feature_Selection/%s_MIs.pkl' % e, 'r') as f:
        selected = pickle.load(f)
    if len(selected)<20: continue
    targets = []
    features = []
    nodes=[]
    for s in selected:
        nodes.append(s[0][:-3])
    nodes = list(set(nodes))
    mask = np.zeros(len(df.columns))
    for n in nodes:
        for i,c in enumerate(df.columns):
            if n == c:
                mask[i] = 1
    H = 145
    for pred_hour_index in range(mat.shape[0]):
        temp_features = []
        if type(pred_hour_index) == slice: continue
        if (pred_hour_index - (lag+H)) >= 0:
            targets.append(mat[pred_hour_index, idx])
            dt = df.index[pred_hour_index-lag]
            cat_features = [work_day_or_holiday(dt), weekday_of_date(dt), dt.hour, dt.month]
#             for i,c in enumerate(mask):
#                 if c == 1:
            for h in np.arange(0,169,24):
                temp_features += [mat[pred_hour_index-(lag+h), idx]]
            temp_features += cat_features
            features.append(temp_features)
    targets = np.array(targets)
    features = np.array(features)
    targets = targets.reshape(-1,1)
    select_idx = np.arange(0,len(targets),1)
    target_24 = targets[select_idx]
    plt.plot(target_24)
    plt.show()
    break
    train_size=0.8
    sample_size=targets.shape[0]
    target_scaler = preprocessing.MinMaxScaler((0,1))
    feature_scaler = preprocessing.MinMaxScaler((0,1))
    scaled_targets = target_scaler.fit_transform(targets)
    scaled_features = feature_scaler.fit_transform(features)
    x = scaled_features
    y = scaled_targets
    train_split = int(train_size*sample_size)
    y_train = y[0:train_split]
    x_train = x[0:train_split,:]
    y_val = y[train_split:]
    x_val = x[train_split:,:]
    model = Sequential()
    model.add(Dense(50, init = 'glorot_uniform', activation = 'tanh', input_dim = x_train.shape[1]))
    model.add(Dense(1, init = 'zero', activation = 'linear'))
    model.compile(loss='mean_squared_error', optimizer='sgd')
    model.fit(x_train, y_train, nb_epoch=20, verbose=0)
    trainPredict = model.predict(x_train)
    testPredict = model.predict(x_val)
    # invert predictions
    trainPredict = target_scaler.inverse_transform(trainPredict)
    trainY = target_scaler.inverse_transform(y_train)
    testPredict = target_scaler.inverse_transform(testPredict)
    testY = target_scaler.inverse_transform(y_val)
    # shift train predictions for plotting
    trainPredictPlot = np.empty_like(y)
    trainPredictPlot[:, :] = np.nan
    trainPredictPlot[:len(trainPredict), :] = trainPredict
    # shift test predictions for plotting
    testPredictPlot = np.empty_like(y)
    testPredictPlot[:, :] = np.nan
    testPredictPlot[len(trainPredict):len(y), :] = testPredict
    # plot baseline and predictions
    plt.plot(target_scaler.inverse_transform(y), label='actual')
    plt.plot(trainPredictPlot, label='train')
    plt.plot(testPredictPlot, label='test')
    plt.legend()
    plt.show()
    print(np.mean(np.abs(testPredict-y[train_split:])/y[train_split:]))
    break


In [41]:
for r in results:
    MAPE = np.mean(np.abs(r[1]-r[2])/r[2])
    print(MAPE)
    if MAPE >= 0.20:
        pass
#         plt.plot(r[2], label='actual')
#         plt.plot(r[1]+5, label='predicted')
#         plt.legend()
#         plt.show()


0.117387148526
0.0473502730337
0.0635781720245
inf
0.0466114307504
0.0635049815157
0.0407133274099
0.0373506136809
0.0467850527958
0.0323876640948
0.0341249367618
0.0418974424136
0.0472132316187
0.0762123123348
0.0858770134741
0.0455078532964
0.0320407730156
0.0321589376366
0.0719348682315
0.15498223998
0.0654836662522
0.0538750811364
0.0413744839967
0.0444361698072
0.0393724085547
0.0364568914879
0.0942110071468
0.118932530345
0.107238975138
0.128426127189
0.156816685434
0.035966704902
0.0691414086909
0.0663116122666
0.078478769951
0.121578093532
0.057574976901
0.0594023429582
0.162882588521
0.150141975579
0.033434361033
0.140488251951
0.0683344627746
0.0275409348353
0.0436281764058
0.0678886532121
0.0300220079275
0.0453456817795
0.0291870396381
0.0594569914868
0.0898635265956
0.0369698090904
0.156929459614
0.127149149874
0.0365958882726
0.109158100287
0.0561388022158
0.0344784084822
0.0675278964333
0.149584521936
0.159429068126
0.111856771036
0.0502541988414
0.0731470485067
0.0450952

In [36]:
for r in results:
    print(r)

('CAPRIDGE_CR1', array([[ 49.13903427],
       [ 51.5670433 ],
       [ 61.4865036 ],
       ..., 
       [ 25.46329308],
       [ 24.08017731],
       [ 22.85133934]], dtype=float32), array([[ 46.84],
       [ 48.93],
       [ 59.57],
       ..., 
       [ 24.19],
       [ 23.17],
       [ 21.86]]), <keras.models.Sequential object at 0x12e28fb90>)
('CAPRIDGE_CR2', array([[ 47.56779099],
       [ 50.35637283],
       [ 59.91588974],
       ..., 
       [ 21.77122879],
       [ 20.37821579],
       [ 19.25756264]], dtype=float32), array([[ 46.84],
       [ 48.93],
       [ 59.57],
       ..., 
       [ 24.19],
       [ 23.17],
       [ 21.86]]), <keras.models.Sequential object at 0x153f1bbd0>)
('CAPRIDGE_R_C', array([[ 42.99233627],
       [ 45.23105621],
       [ 55.06891632],
       ..., 
       [ 21.2408638 ],
       [ 19.80687332],
       [ 18.67268753]], dtype=float32), array([[ 46.84],
       [ 48.93],
       [ 59.57],
       ..., 
       [ 24.19],
       [ 23.17],
       [ 21.86]